# Инференс для оценки экзаменов по русскому языку

Этот notebook позволяет запустить инференс на Google Colab с A100 GPU.

## Шаги:
1. Установка зависимостей
2. Загрузка LoRA адаптера `qwen_sft_exam`
3. Загрузка CSV файла из папки `data`
4. Запуск инференса
5. Сохранение результата

✅ **Все модели загружаются автоматически с HuggingFace!**

## 1. Установка зависимостей

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers accelerate peft bitsandbytes
!pip install -q pandas numpy scikit-learn pillow requests
!pip install -q huggingface_hub

## 2. Импорт библиотек и настройка

In [ ]:
import os
import re
import gc
import random
import logging
import time
from typing import Tuple, List

import numpy as np
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity

from PIL import Image
import requests
from io import BytesIO

from transformers import (
    AutoTokenizer,
    AutoProcessor,
    Qwen2_5_VLForConditionalGeneration,
    BitsAndBytesConfig,
    AutoModel,
)
from peft import PeftModel

# Настройка логирования - ВСЕ логи будут видны в ячейках notebook
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True  # Перезаписываем настройки если они уже были
)
logger = logging.getLogger(__name__)

# Также настраиваем логирование для transformers и других библиотек
logging.getLogger("transformers").setLevel(logging.WARNING)  # Меньше шума от transformers
logging.getLogger("peft").setLevel(logging.WARNING)

print("✅ Логирование настроено - все логи будут выводиться в ячейках notebook")

# Константы
MODEL_NAME = "Qwen/Qwen2.5-VL-3B-Instruct"
ADAPTER_PATH = "qwen_sft_exam"  # Путь к адаптеру (будет загружен в следующей ячейке)
MAX_SEQ_LENGTH = 512
MAX_NEW_TOKENS = 512
REQUEST_TIMEOUT = 3600

print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 3. Загрузка LoRA адаптера `qwen_sft_exam`

Загрузите zip-архив с адаптером `qwen_sft_exam.zip` (или папку `qwen_sft_exam`).  
Если у вас есть архив, распакуйте его. Если папка уже есть - пропустите эту ячейку.

In [ ]:
# Загрузка адаптера
import zipfile
import os

print("=" * 80)
print("📦 ЗАГРУЗКА LoRA АДАПТЕРА")
print("=" * 80)

# Вариант 1: Загрузить zip-архив через Colab интерфейс
from google.colab import files
print("\n📤 Загрузите файл qwen_sft_exam.zip (если есть)")
print("   Или пропустите эту ячейку если папка qwen_sft_exam уже загружена")
uploaded = files.upload()

# Если загружен zip - распаковываем
if uploaded and any(f.endswith('.zip') for f in uploaded.keys()):
    zip_filename = [f for f in uploaded.keys() if f.endswith('.zip')][0]
    print(f"\n📦 Распаковка {zip_filename}...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    print("✅ Адаптер распакован!")
elif os.path.exists('qwen_sft_exam'):
    print("\n✅ Папка qwen_sft_exam уже существует!")
else:
    print("\n⚠️ Папка qwen_sft_exam не найдена. Убедитесь что адаптер загружен.")

# Проверяем наличие адаптера
if os.path.exists('qwen_sft_exam'):
    files_list = os.listdir('qwen_sft_exam')
    print(f"\n✅ Адаптер найден!")
    print(f"   Файлов в адаптере: {len(files_list)}")
    print(f"   Примеры файлов: {files_list[:5]}")
    if 'adapter_model.safetensors' in files_list:
        print("   ✅ adapter_model.safetensors найден - адаптер готов к использованию")
    else:
        print("   ⚠️ adapter_model.safetensors не найден!")
else:
    print("\n❌ ОШИБКА: Адаптер не найден!")
    print("   Загрузите папку qwen_sft_exam или архив qwen_sft_exam.zip")

print("=" * 80)

import threading
import logging
from typing import Tuple

import torch
from transformers import (
    AutoTokenizer,
    AutoProcessor,
    Qwen2_5_VLForConditionalGeneration,
    BitsAndBytesConfig,
    AutoModel,
)
from peft import PeftModel

logger = logging.getLogger(__name__)


MODEL_NAME = "Qwen/Qwen2.5-VL-3B-Instruct"
ADAPTER_PATH = "qwen_sft_exam"


_vl_lock = threading.Lock()
_vl_model = None
_vl_processor = None

_text_lock = threading.Lock()
_text_model = None
_text_tokenizer = None

_ru_lock = threading.Lock()
_ru_model = None
_ru_tokenizer = None


def _bnb_config() -> BitsAndBytesConfig:
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )


def _load_qwen_vl_with_fallback() -> Qwen2_5_VLForConditionalGeneration:
    """Пытаемся загрузить модель в 4-bit; если bitsandbytes без GPU или не доступен CUDA —
    откатываемся на fp16 без квантования."""
    if torch.cuda.is_available():
        try:
            return Qwen2_5_VLForConditionalGeneration.from_pretrained(
                MODEL_NAME,
                quantization_config=_bnb_config(),
                device_map="auto",
                trust_remote_code=True,
                dtype=torch.float16,
                low_cpu_mem_usage=True,
            )
        except Exception as e:
            print("[models] 4-bit загрузка не удалась, fallback на fp16:", e)

    # Fallback: fp16 без bnb
    return Qwen2_5_VLForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        trust_remote_code=True,
        dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True,
    )


def get_vl_model_and_processor() -> Tuple[Qwen2_5_VLForConditionalGeneration, AutoProcessor]:
    global _vl_model, _vl_processor
    if _vl_model is not None and _vl_processor is not None:
        return _vl_model, _vl_processor

    with _vl_lock:
        if _vl_model is not None and _vl_processor is not None:
            return _vl_model, _vl_processor

        logger.info("[models] Загрузка VL модели и процессора...")
        _vl_processor = AutoProcessor.from_pretrained(
            MODEL_NAME,
            use_fast=False,
            trust_remote_code=True,
        )

        _vl_model = _load_qwen_vl_with_fallback()
        _vl_model.eval()
        for p in _vl_model.parameters():
            p.requires_grad = False
        logger.info("[models] VL модель загружена успешно")

    return _vl_model, _vl_processor


def get_text_model_and_tokenizer():
    global _text_model, _text_tokenizer
    if _text_model is not None and _text_tokenizer is not None:
        return _text_model, _text_tokenizer

    with _text_lock:
        if _text_model is not None and _text_tokenizer is not None:
            return _text_model, _text_tokenizer

        logger.info("[models] Загрузка токенизатора и текстовой модели...")
        _text_tokenizer = AutoTokenizer.from_pretrained(
            MODEL_NAME,
            trust_remote_code=True,
            padding_side="left",
            use_fast=False,
        )
        if _text_tokenizer.pad_token is None:
            _text_tokenizer.pad_token = _text_tokenizer.eos_token

        logger.info("[models] Загрузка базовой модели...")
        try:
            if torch.cuda.is_available():
                base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
                    MODEL_NAME,
                    quantization_config=_bnb_config(),
                    device_map="auto",
                    trust_remote_code=True,
                    dtype=torch.float16,
                )
            else:
                raise RuntimeError("CUDA недоступна, используем fp16/32")
        except Exception as e:
            logger.warning(f"[models] 4-bit загрузка базовой модели не удалась, fallback на fp16: {e}")
            base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
                MODEL_NAME,
                device_map="auto",
                trust_remote_code=True,
                dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            )

        logger.info(f"[models] Загрузка LoRA адаптера из {ADAPTER_PATH}...")
        _text_model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
        _text_model.eval()
        for p in _text_model.parameters():
            p.requires_grad = False
        logger.info("[models] Текстовая модель с LoRA загружена успешно")

    return _text_model, _text_tokenizer


def get_rubert_model_and_tokenizer():
    global _ru_model, _ru_tokenizer
    if _ru_model is not None and _ru_tokenizer is not None:
        return _ru_model, _ru_tokenizer

    with _ru_lock:
        if _ru_model is not None and _ru_tokenizer is not None:
            return _ru_model, _ru_tokenizer

        logger.info("[models] Загрузка RuBERT модели...")
        _ru_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2", trust_remote_code=True, use_fast=False)
        _ru_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2", trust_remote_code=True)
        _ru_model.eval()
        for p in _ru_model.parameters():
            p.requires_grad = False
        logger.info("[models] RuBERT модель загружена успешно")

    return _ru_model, _ru_tokenizer




## 4. Функции инференса (из inference.py)

In [ ]:
import os
import re
import gc
import random
import logging
import time
from typing import Tuple, List

import numpy as np
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity

from PIL import Image
import requests
from io import BytesIO

from transformers import BitsAndBytesConfig

# Локальные модели (ленивая загрузка)


logger = logging.getLogger(__name__)


MODEL_NAME = "Qwen/Qwen2.5-VL-3B-Instruct"
ADAPTER_PATH = "qwen_sft_exam"

MAX_SEQ_LENGTH = 512
MAX_NEW_TOKENS = 512
REQUEST_TIMEOUT = 3600


def _filter_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    no_html = re.sub(r"<[^>]+>", "", text)
    # Удаляем латиницу
    result = re.sub(r"[a-zA-Z]", "", no_html)
    return result


def _ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Приводим имена к единым вариантам (не меняем исходные, а только наличие)
    return df


def _load_image_from_url(url: str, timeout: int) -> Image.Image:
    response = requests.get(url, timeout=timeout)
    response.raise_for_status()
    image = Image.open(BytesIO(response.content)).convert("RGB")
    return image


def _generate_image_caption(model, processor, prefix_words: str, url: str) -> str:
    try:
        img = _load_image_from_url(url, timeout=REQUEST_TIMEOUT)
    except Exception as e:
        return f"[Ошибка загрузки: {str(e)}]"

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": f"Опиши изображение (общая длина текста - МЕНЬШЕ 512 символов). Начинай описание с любым из этих слов: {prefix_words}"},
            ],
        }
    ]

    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS)
    caption = processor.decode(outputs[0][inputs["input_ids"].shape[-1]:])
    
    # Освобождаем память от тензоров
    del inputs, outputs
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return caption


def _caption_images(unique_links: List[str]) -> List[str]:
    if len(unique_links) == 0:
        logger.info("[inference] Нет изображений для обработки")
        return []
    logger.info(f"[inference] Начинаем генерацию подписей к {len(unique_links)} изображениям")
    start = time.time()
    model, processor = get_vl_model_and_processor()
    replaces = [
        "На картинке видна",
        "На изоборажении показана",
        "На изображении видна",
        "На картинке изображена",
        "На изображении вы можете увидеть",
        "На данной картинке вы можете увидеть",
    ]
    results = []
    for idx, url in enumerate(unique_links):
        # Логируем каждое изображение
        logger.info(f"[inference] Генерация подписи {idx+1}/{len(unique_links)}: {url[:50]}...")
        word = random.choice(replaces)
        caption = _generate_image_caption(model, processor, word, url)
        results.append(caption)
        
        # Периодическая очистка памяти каждые 5 изображений
        if (idx + 1) % 5 == 0:
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
            if (idx + 1) % 10 == 0:  # Логируем очистку каждые 10 изображений
                logger.debug(f"[inference] Периодическая очистка памяти после {idx + 1} изображений")

    elapsed = time.time() - start
    logger.info(f"[inference] Подписи к изображениям сгенерированы: {len(results)} за {elapsed:.1f} сек ({elapsed/len(unique_links):.1f} сек/изображение)")
    
    # Агрессивная очистка памяти CUDA после завершения цикла
    # Очищаем ссылки на модель и процессор (они глобальные, но освобождаем локальные ссылки)
    del model, processor
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    logger.info("[inference] Память CUDA очищена после генерации подписей, модель и процессор освобождены")
    
    return results


def _summarize_transcription_for_image_tasks(df: pd.DataFrame) -> None:
    # Преобразует поле "Транскрибация ответа" в краткое описание картинки для строк с Тип теста == 1
    if "Тип теста" not in df.columns:
        return
    model, processor = get_vl_model_and_processor()
    
    total_with_images = int(df["Тип теста"].sum()) if "Тип теста" in df.columns else 0
    logger.info(f"[inference] Обработка транскрибаций для {total_with_images} строк с изображениями")
    processed = 0
    start = time.time()

    for i in range(len(df)):
        try:
            if int(df.loc[i, "Тип теста"]) == 1:
                processed += 1
                # Логируем каждые 5 записей или последнюю
                if processed % 5 == 0 or processed == total_with_images or processed == 1:
                    elapsed = time.time() - start
                    eta = (elapsed / processed * (total_with_images - processed)) if processed > 0 else 0
                    logger.info(f"[inference] Сжатие транскрибаций: {processed}/{total_with_images} ({elapsed:.1f} сек, ETA: {eta:.1f} сек)")
                    # Периодическая очистка памяти каждые 10 записей
                    if processed % 10 == 0 and torch.cuda.is_available():
                        torch.cuda.empty_cache()
                text_value = str(df.loc[i, "Транскрибация ответа"]) if "Транскрибация ответа" in df.columns else ""
                messages = [
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": (
                                "На вход тебе дана запись экзамена по русскому языку — описание картинки. "
                                "В качестве ответа верни ТОЛЬКО описание самой картинки. "
                                "Это очень важно для моей карьеры.\n"
                                f"Транскрибация: {text_value}"
                            )}
                        ],
                    }
                ]

                inputs = processor.apply_chat_template(
                    messages,
                    add_generation_prompt=True,
                    tokenize=True,
                    return_dict=True,
                    return_tensors="pt",
                    padding=False,
                ).to(model.device)

                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=MAX_NEW_TOKENS,
                        do_sample=False,
                        use_cache=True,
                        pad_token_id=processor.tokenizer.pad_token_id,
                        eos_token_id=processor.tokenizer.eos_token_id,
                    )

                generated_ids = outputs[0][inputs["input_ids"].shape[-1]:]
                res = processor.decode(generated_ids, skip_special_tokens=True)
                if "Транскрибация ответа" in df.columns:
                    df.loc[i, "Транскрибация ответа"] = res
                
                # Освобождаем память от тензоров после каждой обработки
                del inputs, outputs, generated_ids
        except Exception:
            continue

    # Агрессивная очистка памяти CUDA после завершения цикла обработки транскрибаций
    # Очищаем ссылки на модель и процессор
    del model, processor
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    logger.info("[inference] Память CUDA очищена после сжатия транскрибаций, модель и процессор освобождены")


def _get_sentence_embedding(sentence: str) -> np.ndarray:
    model, tokenizer = get_rubert_model_and_tokenizer()
    encoded = tokenizer(
        sentence if isinstance(sentence, str) else "",
        padding=False,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = model(**encoded)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    result = embeddings[0].numpy()
    # Освобождаем память от тензоров сразу после использования
    del encoded, outputs, embeddings
    # Не удаляем model и tokenizer - они глобальные и используются повторно
    return result


def _semantic_similarity_russian(text1: str, text2: str) -> float:
    emb1 = _get_sentence_embedding(text1).reshape(1, -1)
    emb2 = _get_sentence_embedding(text2).reshape(1, -1)
    similarity = cosine_similarity(emb1, emb2)[0][0]
    return float(similarity)


def _compute_image_similarity(df: pd.DataFrame, unique_links: List[str], captions: List[str]) -> None:
    if "Схожесть описания картинки" not in df.columns:
        df["Схожесть описания картинки"] = 0.0
    link_to_caption = {link: captions[idx] for idx, link in enumerate(unique_links)}
    total_images = len([i for i in range(len(df)) if str(df.loc[i, "Картинка из вопроса"]) != "no image"])
    logger.info(f"[inference] Вычисление семантической схожести для {total_images} записей с изображениями")
    processed = 0
    start = time.time()
    
    for i in range(len(df)):
        try:
            link = str(df.loc[i, "Картинка из вопроса"]) if "Картинка из вопроса" in df.columns else "no image"
            if link and link != "no image" and link in link_to_caption:
                processed += 1
                if processed % 50 == 0 or processed == total_images:
                    elapsed = time.time() - start
                    speed = processed / elapsed if elapsed > 0 else 0
                    eta = (total_images - processed) / speed if speed > 0 else 0
                    logger.info(f"[inference] Вычисление схожести: {processed}/{total_images} ({elapsed:.1f} сек, {speed:.1f} зап/сек, ETA: {eta:.1f} сек)")
                    # Периодическая очистка памяти каждые 100 записей
                    if processed % 100 == 0 and torch.cuda.is_available():
                        torch.cuda.empty_cache()
                        gc.collect()
                        logger.debug(f"[inference] Периодическая очистка памяти после {processed} записей схожести")
                person_text = str(df.loc[i, "Транскрибация ответа"]) if "Транскрибация ответа" in df.columns else ""
                llm_text = link_to_caption[link]
                score = _semantic_similarity_russian(person_text, llm_text)
                df.loc[i, "Схожесть описания картинки"] = score
        except Exception:
            continue
    
    # Агрессивная очистка памяти CUDA после завершения цикла вычисления схожести
    # Модель ruBERT глобальная, но очищаем кеш GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    logger.info("[inference] Память CUDA очищена после вычисления семантической схожести")


def _build_inference_prompt(row: pd.Series) -> str:
    question_num = int(row.get("№ вопроса", 0))
    question_text = str(row.get("Текст вопроса", ""))
    response = str(row.get("Транскрибация ответа", ""))
    test_type = "описание картинки" if int(row.get("Тип теста", 0)) == 1 else "диалог"
    max_score_map = {1: 1, 2: 2, 3: 1, 4: 2}
    max_score = max_score_map.get(question_num, 2)

    prompt = (
        "Ты — эксперт по оценке устных ответов на экзамене по русскому языку для иностранцев.\n"
        "Критерии оценки:\n"
        "1) Ошибки в отдельных словах и единичные несогласованности фраз не считаются ошибкой.\n"
        "2) Тестируемый должен выполнить коммуникативную задачу (должен ответить на вопрос или добиться ответа на свой вопрос).\n"
        "3) Предложения тестируемого должны быть преимущественно полными.\n"
        "Оцени ответ по строгой шкале.\n\n"
        "Контекст:\n"
        f"- № вопроса: {question_num}\n"
        f"- Тип задания: {test_type}\n"
    )

    if int(row.get("Тип теста", 0)) == 1 and pd.notna(row.get("Схожесть описания картинки")):
        similarity = float(row.get("Схожесть описания картинки"))
        prompt += f"- Схожесть описания с изображением: {similarity:.2f}\n"

    prompt += (
        f"- Вопрос: {question_text}\n"
        f"- Ответ кандидата: {response}\n\n"
        f"Оценка (целое число от 0 до {max_score}):"
    )
    return prompt


def _extract_score(text: str, question_num: int) -> int:
    numbers = re.findall(r"\d+", text)
    if not numbers:
        return 0
    score = int(numbers[0])
    max_score = {1: 1, 2: 2, 3: 1, 4: 2}.get(question_num, 2)
    if score < 0:
        return 0
    if score > max_score:
        return max_score
    return score


def _predict_batch(prompts: List[str], question_nums: List[int]) -> List[int]:
    model, tokenizer = get_text_model_and_tokenizer()
    logger.info(f"[inference] Запуск батчевого предсказания для {len(prompts)} примеров")
    start = time.time()
    
    # Токенизация
    logger.info(f"[inference] Токенизация {len(prompts)} промптов...")
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
    ).to(model.device)
    logger.info(f"[inference] Токенизация завершена")

    # Генерация
    logger.info(f"[inference] Запуск генерации модели...")
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    logger.info(f"[inference] Генерация модели завершена")

    # Декодирование
    logger.info(f"[inference] Декодирование результатов...")
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions: List[int] = []
    for i, full_text in enumerate(decoded):
        prompt = prompts[i]
        generated = full_text[len(prompt):].strip()
        score = _extract_score(generated, question_nums[i])
        predictions.append(score)
    
    # Агрессивная очистка памяти от всех больших тензоров
    del inputs, outputs, decoded
    # Освобождаем ссылки на модель и токенизатор (они глобальные, но локальные ссылки удаляем)
    del model, tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    elapsed = time.time() - start
    logger.info(f"[inference] Предсказания завершены, средняя оценка: {np.mean(predictions):.2f} за {elapsed:.1f} сек ({len(prompts)/elapsed:.1f} предсказ/сек)")
    logger.info("[inference] Память CUDA очищена после генерации оценок, модель и токенизатор освобождены")
    return predictions


def run_inference(input_df: pd.DataFrame) -> pd.DataFrame:
    """
    Основной пайплайн инференса. Возвращает DataFrame c добавленной колонкой
    "Оценка экзаменатора" и приведенными вспомогательными полями.
    """
    start_time = time.time()
    logger.info(f"[inference] ========== ЗАПУСК ИНФЕРЕНСА: {len(input_df)} строк ==========")
    df = input_df.copy()

    # Нормализация NaN и подготовка признаков
    logger.info("[inference] Шаг 1/5: Нормализация данных")
    if "Картинка из вопроса" in df.columns:
        df["Картинка из вопроса"] = df["Картинка из вопроса"].fillna("no image")
    else:
        df["Картинка из вопроса"] = "no image"

    # Тип теста: 1 если есть картинка, иначе 0
    df["Тип теста"] = 0
    for idx in range(len(df)):
        try:
            link = str(df.loc[idx, "Картинка из вопроса"]) if "Картинка из вопроса" in df.columns else "no image"
            df.loc[idx, "Тип теста"] = 0 if (not link or link == "no image") else 1
        except Exception:
            df.loc[idx, "Тип теста"] = 0

    # Чистим HTML в "Текст вопроса"
    if "Текст вопроса" in df.columns:
        df["Текст вопроса"] = df["Текст вопроса"].apply(_filter_text)

    # Уникальные ссылки
    saved_links: List[str] = []
    if "Картинка из вопроса" in df.columns:
        for v in df["Картинка из вопроса"].values:
            if isinstance(v, str) and v != "no image" and v not in saved_links:
                saved_links.append(v)

    logger.info(f"[inference] Найдено {len(saved_links)} уникальных изображений, {int(df['Тип теста'].sum())} строк с изображениями")

    # Подписи к изображениям (VL)
    logger.info("[inference] Шаг 2/5: Генерация подписей к изображениям (VL)")
    images_text: List[str] = _caption_images(saved_links)

    # Сжать транскрибации до описания картинки (только для тип теста == 1)
    logger.info("[inference] Шаг 3/5: Сжатие транскрибаций для заданий с картинками")
    _summarize_transcription_for_image_tasks(df)

    # Схожесть описаний
    logger.info("[inference] Шаг 4/5: Вычисление семантической схожести")
    _compute_image_similarity(df, saved_links, images_text)

    # Генерация промптов и предсказаний
    logger.info("[inference] Шаг 5/5: Генерация оценок")
    prompts = df.apply(_build_inference_prompt, axis=1).tolist()
    qnums = [int(v) if pd.notna(v) else 0 for v in df.get("№ вопроса", pd.Series([0] * len(df)))]
    predictions = _predict_batch(prompts, qnums)

    df["Оценка экзаменатора"] = predictions
    
    # Финальная очистка памяти после завершения всего пайплайна
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        logger.info("[inference] Финальная очистка CUDA кеша")
    gc.collect()
    logger.info("[inference] Финальный сбор мусора выполнен")
    
    elapsed = time.time() - start_time
    logger.info(f"[inference] ========== ИНФЕРЕНС ЗАВЕРШЕН: {len(df)} строк обработано за {elapsed:.1f} сек ==========")
    return df




## 5. Загрузка CSV файла из папки `data`

Создайте папку `data` и загрузите туда ваш CSV файл через Colab интерфейс (Files -> Upload), или используйте код ниже для загрузки.

In [ ]:
import os
from google.colab import files

print("=" * 80)
print("📁 ЗАГРУЗКА CSV ФАЙЛА")
print("=" * 80)

# Создаем папку data если её нет
os.makedirs('data', exist_ok=True)
print("\n📂 Папка data/ готова")

# Вариант 1: Загрузить файл напрямую
print("\n📤 Загрузите CSV файл:")
uploaded = files.upload()

if uploaded:
    # Перемещаем файл в папку data
    csv_filename = list(uploaded.keys())[0]
    if not csv_filename.startswith('data/'):
        import shutil
        shutil.move(csv_filename, f'data/{csv_filename}')
        csv_filename = f'data/{csv_filename}'
    print(f"\n✅ Файл загружен: {csv_filename}")
    file_size = os.path.getsize(csv_filename) / 1024 / 1024  # MB
    print(f"   Размер файла: {file_size:.2f} MB")
else:
    # Вариант 2: Использовать файл из папки data
    data_files = [f for f in os.listdir('data') if f.endswith('.csv')]
    if data_files:
        csv_filename = f"data/{data_files[0]}"
        file_size = os.path.getsize(csv_filename) / 1024 / 1024  # MB
        print(f"\n✅ Используется файл из папки data: {csv_filename}")
        print(f"   Размер файла: {file_size:.2f} MB")
    else:
        print("\n❌ ОШИБКА: CSV файл не найден!")
        print("   Загрузите файл в папку data/ или используйте загрузку выше")
        csv_filename = None

print("=" * 80)

## 6. Чтение CSV файла

In [ ]:
print("=" * 80)
print("📖 ЧТЕНИЕ CSV ФАЙЛА")
print("=" * 80)

if csv_filename and os.path.exists(csv_filename):
    # Сначала проверяем первую строку файла
    with open(csv_filename, "r", encoding="utf-8", errors="ignore") as f:
        first_line = f.readline().strip()
        print(f"Первая строка файла: {first_line[:100]}...")
    
    # Если первая строка содержит ';', значит разделитель точно ';'
    if ';' in first_line:
        sep = ';'
        print(f"✅ Определен разделитель: '{sep}' (по первой строке)")
    else:
        # Пробуем определить автоматически
        delimiters = [';', ',', '\t']
        with open(csv_filename, "r", encoding="utf-8", errors="ignore") as f:
            lines = [f.readline() for _ in range(3)]
        
        delimiter_scores = {}
        for delim in delimiters:
            counts = [line.count(delim) for line in lines if line.strip()]
            if counts:
                delimiter_scores[delim] = sum(counts) / len(counts)
        
        if delimiter_scores:
            sep = max(delimiter_scores, key=delimiter_scores.get)
            print(f"✅ Определен разделитель: '{sep}' (автоопределение)")
        else:
            sep = ';'
            print(f"⚠️ Используется разделитель по умолчанию: '{sep}'")
    
    # Читаем CSV с правильным разделителем
    try:
        df = pd.read_csv(csv_filename, sep=sep, encoding='utf-8', on_bad_lines='skip', engine='python')
        
        # Проверяем что колонки правильно разделены
        if len(df.columns) == 1 and ';' in str(df.columns[0]):
            print("⚠️ Колонки не разделены правильно, пробуем исправить...")
            # Разделяем первую колонку если она содержит все названия
            first_col = df.columns[0]
            if ';' in str(first_col):
                # Читаем заново с правильным разделителем
                df = pd.read_csv(csv_filename, sep=';', encoding='utf-8', on_bad_lines='skip', engine='python')
        
        print(f"✅ Загружено {len(df)} строк")
        print(f"✅ Колонок: {len(df.columns)}")
        print(f"Колонки: {list(df.columns)}")
        
        # Проверяем что колонки правильные
        expected_cols = ['Id экзамена', 'Id вопроса', '№ вопроса', 'Текст вопроса', 
                        'Картинка из вопроса', 'Оценка экзаменатора', 'Транскрибация ответа']
        found_cols = [col for col in expected_cols if col in df.columns]
        if len(found_cols) >= 3:
            print(f"✅ Найдены необходимые колонки: {found_cols}")
        else:
            print(f"⚠️ ВНИМАНИЕ: Не все необходимые колонки найдены!")
            print(f"Ожидались: {expected_cols}")
            print(f"Найдены: {list(df.columns)}")
            
    except Exception as e:
        print(f"❌ Ошибка при чтении CSV: {e}")
        # Пробуем другие разделители
        for alt_sep in [';', ',', '\t']:
            if alt_sep != sep:
                try:
                    print(f"Пробуем разделитель '{alt_sep}'...")
                    df = pd.read_csv(csv_filename, sep=alt_sep, encoding='utf-8', on_bad_lines='skip', engine='python')
                    if len(df.columns) > 1:
                        print(f"✅ Загружено {len(df)} строк с разделителем '{alt_sep}'")
                        print(f"Колонки: {list(df.columns)}")
                        sep = alt_sep
                        break
                except Exception as e2:
                    continue
        else:
            print(f"❌ КРИТИЧЕСКАЯ ОШИБКА: Не удалось прочитать CSV файл")
            df = None
    
    if df is not None:
        if len(df) == 0:
            print("⚠️ ВНИМАНИЕ: DataFrame пустой!")
            df = None
        elif len(df.columns) == 1:
            print("⚠️ ВНИМАНИЕ: Только одна колонка! Файл не распарсен правильно.")
            print("Попробуйте указать разделитель вручную или проверьте формат файла.")
else:
    print("❌ ОШИБКА: CSV файл не найден!")
    df = None

## 7. Запуск инференса

⚠️ **ВНИМАНИЕ**: Инференс может занять много времени (от нескольких минут до часов в зависимости от размера файла).

**Логи будут выводиться здесь, в этой ячейке**, под кодом. Вы увидите:
- Прогресс каждого этапа (1/5, 2/5, и т.д.)
- Количество обработанных записей
- Примерное время до завершения (ETA)

In [ ]:
# Запуск инференса
if df is not None:
    print("=" * 80)
    print("🚀 НАЧАЛО ИНФЕРЕНСА")
    print("=" * 80)
    print(f"📊 Обрабатывается файл: {csv_filename}")
    print(f"📈 Всего строк для обработки: {len(df)}")
    print(f"📋 Колонок в файле: {len(df.columns)}")
    print("\n⏳ Инференс запущен, логи будут выводиться ниже...")
    print("=" * 80)
    print()
    
    # Запускаем инференс - все логи будут видны здесь
    result_df = run_inference(df)
    
    print()
    print("=" * 80)
    print("✅ ИНФЕРЕНС ЗАВЕРШЕН УСПЕШНО!")
    print("=" * 80)
    print(f"\n📊 Результаты:")
    print(f"   • Всего строк обработано: {len(result_df)}")
    print(f"   • Средняя оценка: {result_df['Оценка экзаменатора'].mean():.2f}")
    print(f"   • Распределение оценок:")
    score_counts = result_df['Оценка экзаменатора'].value_counts().sort_index()
    for score, count in score_counts.items():
        percentage = (count / len(result_df)) * 100
        print(f"     - Оценка {score}: {count} записей ({percentage:.1f}%)")
    print("=" * 80)
else:
    print("❌ ОШИБКА: DataFrame не загружен!")
    print("Проверьте что CSV файл был правильно загружен и распарсен.")
    result_df = None

## 8. Сохранение результата

In [ ]:
# Сохранение результата
print("=" * 80)
print("💾 СОХРАНЕНИЕ РЕЗУЛЬТАТА")
print("=" * 80)

if result_df is not None:
    output_filename = "result_with_scores.csv"
    
    # Сохраняем в корневую папку
    result_df.to_csv(output_filename, index=False, sep=';', encoding='utf-8')
    file_size = os.path.getsize(output_filename) / 1024 / 1024  # MB
    print(f"\n✅ Результат сохранен в файл: {output_filename}")
    print(f"   Размер файла: {file_size:.2f} MB")
    
    # Также сохраняем в папку data
    data_output = f"data/{output_filename}"
    result_df.to_csv(data_output, index=False, sep=';', encoding='utf-8')
    print(f"\n✅ Результат также сохранен в: {data_output}")

    # Скачать файл
    print(f"\n📥 Начинается скачивание файла...")
    from google.colab import files
    files.download(output_filename)
    print("✅ Файл готов к скачиванию!")
    print("\n🎉 ВСЕ ГОТОВО! Результаты сохранены и готовы к скачиванию.")
else:
    print("\n❌ ОШИБКА: Нет результатов для сохранения!")
    print("   Проверьте что инференс завершился успешно.")

print("=" * 80)